In [1]:
!pip install rdkit

In [2]:
from rdkit import Chem

In [3]:
def is_bridgehead_double_bond_to_carbon(mol, bridgehead_atoms):
    double_bond_atoms = []
    for atom_idx in bridgehead_atoms:
        atom = mol.GetAtomWithIdx(atom_idx)
        for bond in atom.GetBonds():
            if bond.GetBondType() == Chem.rdchem.BondType.DOUBLE:
                other_atom = bond.GetOtherAtom(atom)
                if other_atom.GetAtomicNum() == 6 and other_atom.IsInRing():
                    double_bond_atoms.append(atom_idx)
    return double_bond_atoms


def classify_molecule(smiles):
    mol = Chem.MolFromSmiles(smiles)
    ssr = Chem.GetSymmSSSR(mol)
    rings = [set(ring) for ring in ssr]
    if len(rings) < 2:
        return "neither"

    bridgehead_atoms = []
    for i, ring1 in enumerate(rings):
        for ring2 in rings[i + 1:]:
            shared_atoms = ring1 & ring2
            if len(shared_atoms) == 2:
                for atom in shared_atoms:
                    neighbors = mol.GetAtomWithIdx(atom).GetNeighbors()
                    if any(neighbor.GetIdx() in shared_atoms for neighbor in neighbors):
                        return "fused"
            elif len(shared_atoms) > 2:
                bridgehead_atoms = list(shared_atoms)
                double_bond_atoms = is_bridgehead_double_bond_to_carbon(mol, bridgehead_atoms)
                if double_bond_atoms:
                    for atom_idx in double_bond_atoms:
                        smallest_ring_size = min(len(ring) for ring in ssr if atom_idx in ring)
                        if smallest_ring_size < 8:
                            return "anti-Bredt"
                return "bridged"

    return "neither"

mol1 = ('C1C[C@H]2C[C@@H]1C=C2')  #Bicyclo[2.2.1]hept-2-ene  bridged
mol2 = ('C1CC2C1CC=C2')  #Bicyclo[3.2.0]hept-2-ene  fused
mol3 = ('CC1=C2CC[C@@]2([C@H]3CC(C[C@H]3C1)(C)C)C')  #Delta(6)-protoilludene   fused
mol4 = ('CC1=C2C[C@](C[C@H]2CC3=COC(=C13)C(=O)O)(C)CO') #Tsugicoline fused
mol5 = ('C[C@@H]1CC[C@@H]2CC3=C(CC[C@]13C2(C)C)C') #Cyperene bridged
mol6 = ('O=C1C2CCCCC1CC2') #Bicyclo(4.2.1)nonan-9-one bridged
mol7 = ('O=C1CCC2CC1CCC2=O') # Bicyclo(3.3.1)nonane-2,6-dione bridged
mol8 = ('O=C1C2CC3CC1CC(C2)C3=O') #2,6-Adamantandione bridged
mol9 = ('C1CC2CCC1=C2') #bridged
mol10 = ('CC1=C2C[C@](C[C@H]2CC3=COC(=C13)C(=O)O)(C)CO') #fused
mol11 = ('C1CCC2CCCCC(C1)C2') #bridged
mol12 = ('C1C=C2CC1C2')#Bicyclo[2.1.1]hexene bridged
mol13 = ('C1C2C1=C2') #Bicyclo[1.1.0]butene fused
mol14 = ('C1CC=C2CC=CCC(=C1)C2')#Bicyclo[4.4.1]undeca-1,5,8-triene bridged violate
mol15 = ('O=C1C=C2[C@](C)(CC1)[C@H]3CC[C@]4(C)[C@H](CC[C@H]4[C@@H]3CC2)O') #fused
mol15 = ('C1C2C=CC1C=C2')#Norbornadiene
   
classify_molecule('C1CC=C2CC=CCC(=C1)C2')

'anti-Bredt'